In [ ]:
from utils_classification import *

# Load data

## MNIST

In [ ]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

mnist_folder = os.path.join('data', 'MNIST')

train_images_path = os.path.join(mnist_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(mnist_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(mnist_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(mnist_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

## Segmented cards from task 0

In [ ]:
filepath_task0 = os.path.join('data', 'classification_data', 'all_games_classification_series.pickle')
df_task0, df_numbers_only, df_not_numbers = load_segmentation_task0(filepath_task0)

## Data loader and augmentation

In [ ]:
transform_aug = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.RandomAffine(degrees=25, scale=(0.8, 1.1), shear=20),
                transforms.ToTensor(),
            ]
        )

augmented_train = augment_data(train_images, transform_aug)

In [ ]:
# Parameters of our DataLoader
params = {'batch_size': 128,
          'shuffle': True}

# Creation of a train/test dataset & dataloader
train_merged_images = np.concatenate((train_images, augmented_train))
train_merged_labels = np.concatenate((train_labels, train_labels))

ds_train = MNISTDataset(train_merged_images, train_merged_labels, transform=transforms.ToTensor())
dl_train = data.DataLoader(ds_train, **params)

ds_test = MNISTDataset(test_images, test_labels, transform=transforms.ToTensor())
dl_test = data.DataLoader(ds_test, **params)

In [ ]:
task0_numbers = np.array([v for v in df_numbers_only.image.apply(preprocess_segmented_task0)])

ds_test_task0 = MNISTDataset(task0_numbers, df_numbers_only['rank'].to_numpy().astype(np.int64), 
                             transform=transforms.ToTensor())
dl_test_task0 = data.DataLoader(ds_test_task0, **params)

# Training and testing

## Models

In [ ]:
model1 = Net1()
model2 = Net2()
model3 = Net3()

## Training

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
learning_rate = 1e-3
opt = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
epochs = 7

# Train & test our model until convergence
for e in range(epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_loop(dl_train, model2, criterion, opt)
    test_loop(dl_test, model2, criterion)

In [ ]:
test_loop(dl_test_task0, model2, criterion)